In [1]:
import os
import glob
import json
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
import imgaug as ia
from imgaug import augmenters as iaa

In [3]:
valueTransforms = ['gaussianblur_1',
 'gaussianblur_10',
 'gaussianblur_20',
 'superpixels_0p1',
 'superpixels_0p5',
 'superpixels_0p85',
 'colorspace_25',
 'colorspace_50',
 'averageblur_5_11',
 'medianblur_1',
 'sharpen_0',
 'sharpen_1',
 'sharpen_2',
 'addintensity_-80',
 'addintensity_80',
 'elementrandomintensity_1',
 'multiplyintensity_0p25',
 'multiplyintensity_2',
 'contrastnormalization_0',
 'contrastnormalization_1',
 'contrastnormalization_2',
 'elastic_1',
 'invert']
spaceTransforms2 = ['scaled_1p25',
 'scaled_0p75',
 'scaled_0p5',
 'scaled_(1p25, 1p0)',
 'scaled_(0p75, 1p0)',
 'scaled_(1p0, 1p25)',
 'scaled_(1p0, 0p75)',
 'translate_(0p1, 0p1)',
 'translate_(0p1, -0p1)',
 'translate_(-0p1, 0p1)',
 'translate_(-0p1, -0p1)',
 'translate_(0p1, 0)',
 'translate_(-0p1, 0)',
 'translate_(0, 0p1)',
 'translate_(0, -0p1)',
 'rotated_3',
 'rotated_5',
 'rotated_10',
 'rotated_45',
 'rotated_60',
 'rotated_90',
 'flipH',
 'flipV',
 'dropout',
 'piecewiseAffine_0p01',#
 'piecewiseAffine_0p03',#
 'piecewiseAffine_0p06',#
 'piecewiseAffine_0p1']#
len(spaceTransforms2)

28

In [4]:
##Space transforms
spaceTransforms = {}
# Change image scale keeping aspect ratio same ( same scaling of width and height)
for r in [1.25,.75,.5]:
    spaceTransforms['scaled_' + str(r).replace('.','p')] = iaa.Sequential([
        iaa.Affine(scale=r)
    ])
# Change image scale not keeping aspect ratio same ( unequal scaling of width and height)
for r in [(1.25,1.0), (.75,1.0), (1.0,1.25), (1.0,.75)]:#125% or 75% of one of the dimensions
    spaceTransforms['scaled_' + str(r).replace('.','p')]=iaa.Sequential([
        iaa.Affine(scale={"x": r[0], "y": r[1]})
    ])
    
# translate image by +-10% towards each of corners(diagonally) or edges(horizontally and vertically)
for tr in [(.1,.1), (.1,-.1), (-.1,.1), (-.1,-.1), (.1,0), (-.1,0), (0,.1), (0,-.1)]:
    spaceTransforms['translate_' + str(tr).replace('.','p')]=iaa.Sequential([
        iaa.Affine(translate_percent={"x":tr[0], "y":tr[1]})
    ])
    
# Change orientation by few small angles and few large angles
for theta in [3,5,10,45,60,90]:
    spaceTransforms['rotated_' + str(theta).replace('.','p')]=iaa.Sequential([
        iaa.Affine(rotate=theta)
    ])
    
# Flip images horizontly
for _ in [1]:
    spaceTransforms['flipH']=iaa.Sequential([
        iaa.Fliplr(1.0)
    ])
    
# Flip images vertically
for _ in [1]:
    spaceTransforms['flipV']=iaa.Sequential([
        iaa.Flipud(1.0)
    ])
# Random pixel dropout, 10% of pixels, like salt and pepper noise without salt
for _ in [1]:
    spaceTransforms['dropout'] = iaa.Sequential([
        iaa.Dropout(p=0.1)
    ])
# Good local distortions, Refer at http://imgaug.readthedocs.io/en/latest/source/augmenters.html#piecewiseaffine
#for z in [0.01,0.03,0.06,0.1]:
#    spaceTransforms['piecewiseAffine_'+str(z).replace('.','p')]=iaa.Sequential([
#        iaa.PiecewiseAffine(scale=z)
#    ])
len(spaceTransforms)

28

In [5]:
#Files to read
models = ['e2e_faster_rcnn_R-50-C4_2x',
'e2e_faster_rcnn_R-50-FPN_2x',
'e2e_faster_rcnn_R-101-FPN_2x',
'e2e_faster_rcnn_X-101-64x4d-FPN_2x',
'e2e_mask_rcnn_R-50-C4_2x',
'e2e_mask_rcnn_R-50-FPN_2x',
'e2e_mask_rcnn_R-101-FPN_2x',
'retinanet_R-50-FPN_2x',
'retinanet_R-101-FPN_2x',
'retinanet_X-101-64x4d-FPN_2x']
dataF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/'
outputs = dataF+'outputs/'

resF = outputs +'5000_original_results/'
tranF = outputs + 'transformed_outputs/'
spaceF = outputs + 'transformed_spatial_output/'
newResF = outputs +'5000_original_results_matches/'
newTranF = outputs + 'transformed_outputs_matches/'
newSpaceF = outputs + 'transformed_spatial_output_matches/'
#create new directories
pathsV = [newTranF+_+'/' for _ in valueTransforms]
pathsS = [newSpaceF+_+'/' for _ in spaceTransforms]
for path in [newResF]+pathsV+pathsS:
    for algo in models:
        directory = os.path.dirname(path+algo+'/')
        if not os.path.exists(directory):
            os.makedirs(directory)
#COCO annotations file
anntF = dataF+'inputs/annotations/instances_val2017.json'


In [6]:
cocoGt=COCO(anntF)
#cocoDt=cocoGt.loadRes(origF+algoF + '0_e2e_faster_rcnn_R-50-C4_2x.json')

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


In [7]:
#imgIds = cocoGt.getImgIds()
annIds = cocoGt.getAnnIds(imgIds = [530162])
anns = cocoGt.loadAnns(annIds)
dt = pd.DataFrame(anns)
dt


,area,bbox,category_id,id,image_id,iscrowd,segmentation
0,28193.59970,"[126.72, 18.22, 381.43, 218.56]",28,283003,530162,0,"[[318.49, 26.22, 318.49, 18.78, 321.24, 18.22,..."
1,882.09260,"[0.24, 156.89, 28.77, 66.78]",28,284085,530162,0,"[[0.42, 156.89, 6.39, 160.33, 13.09, 165.58, 1..."
2,13106.33585,"[307.06, 271.55, 131.45, 154.49]",1,1230651,530162,0,"[[430.84, 420.28, 307.06, 426.04, 313.77, 366...."
3,14034.33930,"[176.56, 274.43, 134.33, 147.77]",1,1255402,530162,0,"[[308.02, 421.24, 180.4, 422.2, 176.56, 397.25..."
4,9113.10595,"[534.53, 310.58, 104.76, 115.51]",1,1277571,530162,0,"[[556.56, 379.35, 552.8, 370.75, 550.65, 365.3..."
5,7465.50055,"[0.25, 142.7, 54.64, 244.7]",1,1282917,530162,0,"[[10.13, 236.28, 10.71, 214.19, 14.78, 194.43,..."
6,16278.32565,"[139.45, 165.9, 111.56, 255.81]",1,1697952,530162,0,"[[178.88, 171.67, 165.41, 187.05, 163.49, 204...."
7,5668.68900,"[396.17, 283.41, 69.27, 143.59]",1,1700528,530162,0,"[[396.17, 319.32, 400.28, 312.65, 400.28, 303...."
8,491.92805,"[141.2, 187.23, 20.89, 37.98]",1,1702706,530162,0,"[[151.33, 225.21, 159.56, 210.65, 162.09, 212...."
9,15130.11685,"[265.8, 145.85, 151.6, 218.78]",1,1704618,530162,0,"[[293.62, 303.22, 265.8, 264.84, 270.59, 247.5..."


In [8]:
def bbMatch(A,B, base = 'union'):
    '''Returns a Jaccard like score of match between the two bounding boxes. IntersectionArea / BaseArea'''
    areaA = A[2]*A[3]#First object's area
    areaB = B[2]*B[3]#Second objects area
    #print(areaU)
    if areaA==0 or areaB ==0:
        return 0#avoid division by zero when no match
    if B[2]<0 or B[3]<0:
        print('less than zero ', B)
    xa1,ya1,xa2,ya2 = A[0],A[1],A[0]+A[2], A[1]+A[3]
    xb1,yb1,xb2,yb2 = B[0],B[1],B[0]+B[2], B[1]+B[3]
    dx = min(xa2, xb2) - max(xa1, xb1)#overlap in x
    dy = min(ya2, yb2) - max(ya1, yb1)#overlap in y
    areaI= dx*dy if (dx>=0) and (dy>=0) else 0
    #Possible base areas
    area = {'first':   areaA,
            'second':  areaB,
            'sum':     areaA+areaB,
            'union':   areaA+areaB-areaI,#union = sum - intersection
            'larger':  areaA if areaA>areaB else areaB,
            'smaller': areaA if areaA<areaB else areaB}
    
    return areaI/area[base];
A = [415.11, 204.57, 172.01, 258.54]
B = [423.078522, 205.677017, 580.738586, 441.871582]
bbMatch(A,B)

0.16314116498247844

In [9]:
def findMatchingBBoxes(detectedJsons, cocoAnnotations,outDir,algo):
    '''loopes over all images in detected Jsons.
    For each object in these images, finds the best matching object
    provided in coco Annotations for this particular image.
    Match is done by comparing overlapping area (Jaccard measure) of the two bounding boxes.
    Must match atleast 10%.
    Also creates a new json file that is more structured'''
    #outputs = []
    for i, js in enumerate(detectedJsons):#multiple json files returned by glob
        #print(js)
        if i>0:#for now to save time read only one file. Remove in actual script or parallelize
            break
        with open(js) as fd:
            imgs = json.load(fd)#go over each file
        outputs = []#[None]*len(imgs)
        for _idx,img in enumerate(imgs):#array of info
            img_name = img['img_name'].split('/')[-1]
            parts = img_name.split('__')
            #print(parts)
            if len(parts)==2:
                transform = parts[0]
                imgId = int(parts[1][:-4])
                if 'piecewise' in transform:#skip all piecewise
                    continue
            else:
                imgId = int(parts[0][:-4])
                transform = None
            detected = []
            count = 0
            uniques = set()
            dt = pd.DataFrame(cocoAnnotations.loadAnns(cocoGt.getAnnIds(imgIds = [imgId])))
            for index,A in enumerate(img['bboxes']):#for each object detected by NN
                #reframe information to write in json
                info ={'detClass':img['classes'][index],
                            'detBbox':A,
                            'score':img['scores'][index],
                            'matchObjectId':None,
                            'matchClass':None,
                            'matchBBox':None,}
                detected.append(info)
                if img['scores'][index]<0.5:#skip objects with low confidence
                    continue
                best, bestM = None,-1
                for _, obj in dt.iterrows():#go over all given annotations for this image
                    B = obj.bbox
                    #print(B, transform)
                    if transform in spaceTransforms:#need to transform bbox and transform != 'dropout'
                        bbs = ia.BoundingBoxesOnImage(#make from x,y,w,h
                            [ia.BoundingBox(x1=B[0], y1=B[1], x2=B[0]+B[2], y2=B[1]+B[3])],
                            shape=[cocoGt.imgs[imgId]['height'], cocoGt.imgs[imgId]['width']])
                        T = spaceTransforms[transform]
                        try:
                            B = T.augment_bounding_boxes([bbs])[0].bounding_boxes[0]
                            B = [B.x1, B.y1, B.x2-B.x1, B.y2-B.y1]
                        except:
                            print(B,transform)
                            #B = obj.bbox
                            continue
                        #print(B)
                    #Use larger as base of match, for better result as prevents saturation (match=1)
                    m = bbMatch(A,B, 'larger')#find match between two objects' bboxes
                    if m >=0.10 and m>bestM:#best of all matches with atleast 10% match
                        best,bestM = obj,m#id defined by coco annotations
                if not best is None:#preserve order
                    info['matchObjectId']=best.id
                    info['matchClass']=best.category_id
                    info['matchBBox']=best.bbox
                    uniques.add(best.id)
                    count = count+1
            #matchedBBoxes[str(imgId)] = matches
            outputs.append({'image_id':imgId,
                             'objects':detected,#all objects detected in this image
                            'totalDetected': len(img['bboxes']),
                            'matched':count,
                            'uniqueMatches':len(uniques),
                            'transform':transform})
        #Divide results into separate transformations, once per json file
        
        dt = pd.DataFrame(outputs)
        trsfms = dt['transform'].unique()
        print(trsfms)
        if len(trsfms) == 1 and trsfms[0] is None:#from orig 5000 results
            path = outDir + algo+'/'+os.path.basename(js)
            with open(path,'w') as fd:
                #print(path)
                fd.write(json.dumps(dt.to_dict('index')))
        else:#a transformed result
            for tr in trsfms:
                path = outDir + tr + '/' + algo+'/'+os.path.basename(js)
                with open(path,'w') as fd:
                    #print(path)
                    fd.write(json.dumps(dt[dt['transform']==tr].to_dict('index')))
    #return outputs

In [10]:
for algo in models[0:1]:
    origJsons = glob.iglob(resF+algo + '/*.json')
    #findMatchingBBoxes(origJsons, cocoGt,newResF,algo)
    origJsons = glob.iglob(tranF+algo + '/*.json')
    #findMatchingBBoxes(origJsons, cocoGt,newTranF,algo)
    origJsons = glob.iglob(spaceF+algo + '/*.json')
    #print(spaceF+algo)
    findMatchingBBoxes(origJsons, cocoGt,newSpaceF,algo)

#matches

[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[208.2, 254.04, 5.61, 1.28] scaled_0p75
[129.4, 52.38, 3.05, 1.53] scaled_0p5
[129.4, 52.38, 3.05, 1.53] scaled_0p5
[129.4, 52.38, 3.05, 1.53] scaled_0p5
[129.4, 52.38, 3.05, 1.53] scaled_0p5
[129.4, 52.38, 3.05, 1.53] scaled_0p5
[129.4, 52

0 100 0 100
